In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train.shape

(42000, 785)

In [7]:
test.shape

(28000, 784)

In [9]:
train_y = train['label'].astype('float32')
train_x = train.drop(['label'], axis=1).astype('int32')
test_x = test.astype('float32')
train_x.shape, train_y.shape, test_x.shape

((42000, 784), (42000,), (28000, 784))

In [10]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x / 255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x / 255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [11]:
#One hot encoding

train_y = tf.keras.utils.to_categorical(train_y, 10)
train_y.shape

(42000, 10)

In [12]:
train['label'].head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [13]:
train_y[0:5,:]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [16]:
#Defining the model

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
    tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = (2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu', padding = 'Same'),
    tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = (2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0

In [17]:
#Defining the callback function to stop our training once the aceeptable accuracy is reached

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.999):
            print('Reached 99.9% accuracy so cancelling training')
            self.model.stop_traning = True
            
callbacks = myCallback()

In [18]:
#Compiling and model training with batch size=50, epochs=20 and optimizer=adam

Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.0005,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=50, epochs=20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 101s 115ms/step - loss: 0.4672 - accuracy: 0.8427
Epoch 2/20
840/840 [==============================] - 95s 113ms/step - loss: 0.1053 - accuracy: 0.9708
Epoch 3/20
840/840 [==============================] - 94s 112ms/step - loss: 0.0748 - accuracy: 0.9791
Epoch 4/20
840/840 [==============================] - 95s 114ms/step - loss: 0.0618 - accuracy: 0.9828
Epoch 5/20
840/840 [==============================] - 93s 110ms/step - loss: 0.0491 - accuracy: 0.9865
Epoch 6/20
840/840 [==============================] - 92s 109ms/step - loss: 0.0427 - accuracy: 0.9875
Epoch 7/20
840/840 [==============================] - 92s 109ms/step - loss: 0.0377 - accuracy: 0.9895
Epoch 8/20
840/840 [==============================] - 91s 108ms/step - loss: 0.0344 - accuracy: 0.9901
Epoch 9/20
840/840 [==============================] - 91s 109ms/step - loss: 0.0283 - accuracy: 0.9915s - loss: 0.0279 - accu
Epoch 10/20
840/840 [============================

In [19]:
results = model.predict(test_x)

results = np.argmax(results, axis = 1)
results = pd.Series(results, name = 'Label')

In [20]:
submission = pd.concat([pd.Series(range(1,28001), name ='Imageid'),  results], axis=1)
submission.to_csv('submission.csv',index=False)